<a href="https://colab.research.google.com/github/engcarlosperezmolero/tus_clases_automatic_response_scraper/blob/main/tus_clases_automatic_response_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
sys.path.insert(0,'/content/drive/My Drive/credentials')

from tusclases_secrets import credentials # in drive I've created a folder called credentials with a file called tusclases_secrets.py inside.

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

options = webdriver.ChromeOptions() # Usamos chrome, se podria usar otro.
options.add_argument('--headless') # Chromium sin interfaz grafica
options.add_argument('--no-sandbox') # Seguridad
options.add_argument('--disable-dev-shm-usage') # configuracion de linux
options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent

driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
cuerpo_mensaje_automatizado = "Hola, este es un mensaje automatizado. En caso de que necesites una respuesta inmediata me puedes escribir a mi WhatsApp: +5491122510584"
driver.get("https://www.tusclases.com.ar/area-profesores/messaging")
time.sleep(2)
driver.set_window_size(4096, 2160)
driver.save_screenshot("screenshot_tusclases_0.png")
try:
    input_email = driver.find_element(by=By.XPATH, value="/html/body/div[2]/form/div[3]/div/div/div/div[2]/div[2]/div[1]/div[2]/div/div/div[1]/input")
    input_password = driver.find_element(by=By.XPATH, value="/html/body/div[2]/form/div[3]/div/div/div/div[2]/div[2]/div[1]/div[2]/div/div/div[2]/input")
    input_email.send_keys(credentials.tus_clases_user)
    input_password.send_keys(credentials.tus_clases_password + Keys.ENTER)
    time.sleep(2)
    driver.save_screenshot("screenshot_tusclases_1.png")

except NoSuchElementException as e:
    print(e)
finally:
    mensajes_no_leidos = driver.find_elements(by=By.XPATH, value="/html/body/div[2]/form/div[3]/div/div/div/div[1]/div[3]/*[@class='rc unread']")
    mensajes = driver.find_elements(by=By.XPATH, value="/html/body/div[2]/form/div[3]/div/div/div/div[1]/div[3]/*[contains(@class, 'rc')]")

    if len(mensajes_no_leidos) == 0:
        for num_mensaje, mensaje in enumerate(mensajes):
            # solo itera los 5 primeros mensajes
            if num_mensaje == 5:
                break
                
            mensaje.click()
            time.sleep(2)
            mensajes_enviados_por_mi = driver.find_elements(by=By.XPATH, value="/html/body/div[2]/form/div[3]/div/div/div/div[2]/div[3]/*[contains(@class, 'conb me')]/*[@class='lin']")
            texto_total_enviado = " | ".join([mensaje.get_attribute("innerHTML") for mensaje in mensajes_enviados_por_mi])

            if len(mensajes_enviados_por_mi) == 0 or "+5491122510584" not in texto_total_enviado:
                text_area = driver.find_element(by=By.XPATH, value="/html/body/div[2]/form/div[3]/div/div/div/div[2]/div[4]/div/div[1]/div/textarea")
                text_area.send_keys(cuerpo_mensaje_automatizado + Keys.TAB + Keys.ENTER)

    else:
        for mensaje_no_leido in mensajes_no_leidos:
            mensaje_no_leido.click()
            time.sleep(2)
            text_area = driver.find_element(by=By.XPATH, value="/html/body/div[2]/form/div[3]/div/div/div/div[2]/div[4]/div/div[1]/div/textarea")
            text_area.send_keys(cuerpo_mensaje_automatizado + Keys.TAB + Keys.ENTER)


In [ ]:
driver.save_screenshot("screenshot_tusclases_2.png")
driver.refresh()
time.sleep(2)
driver.save_screenshot("screenshot_tusclases_refreshed.png")

True